<a href="https://colab.research.google.com/github/danielbryksil/Calculadora-LF-Secund-rio/blob/main/Calculadora_de_Carteira_LF_CDI_%2B.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Calculadora PU CDI +


In [1]:
!pip install python-bcb
!pip install pandas_market_calendars

In [2]:
from datetime import datetime
import pandas as pd
from bcb import sgs
import numpy as np
from datetime import datetime, timedelta
from google.colab import files
import pandas as pd
import requests

In [3]:
dados_excel = pd.read_excel('/content/feriados.xlsx')
feriados = dados_excel['2001-01-01'].tolist()

In [4]:
df = pd.read_excel('/content/lf secun python cdi +.xlsx')
df

,Tipo Aplic.,Data Inicial,Data de Vencimento,Emissor,Banco,Valor Inicial,Valor Aplicado,Taxa COMPRA,Transação,Titulo,Data Emissão,Data Vencimento,PU Inicial,Taxa de Emissão,Taxa de Venda,Quantidade
0,LF,2023-04-18,2024-05-06,SANTANDER,SANTANDER,4000000,5000000,0.008,1000000007375,LF0021005P4,2021-05-06,2024-05-06,50000,0.00750,0.003,75
1,LF,2023-05-08,2024-06-03,ITAU,ITAU,4000000,5000000,0.008,1000000007437,LF0022009MZ,2022-03-28,2024-06-03,100000,0.00658,0.002,41
2,LF,2023-05-08,2024-06-03,ITAU,ITAU,4000000,5000000,0.008,1000000007438,LF0022001K1,2022-01-21,2024-06-03,100000,0.00667,0.002,40
3,LF,2023-07-26,2024-07-08,SICREDI,SICREDI,4000000,5000000,0.015,1000000007519,LF002200J39,2022-07-07,2024-07-08,150000,0.01150,0.004,27
4,LF,2023-08-02,2024-07-22,SICREDI,SICREDI,4000000,5000000,0.015,1000000007534,LF0021009WQ,2021-07-22,2024-07-22,150000,0.01550,0.004,25


In [5]:
data_operação = input("Qual a data da operação (aaaa-mm-dd)? ")

Qual a data da operação (aaaa-mm-dd)? 2024-02-27


In [6]:
data_str = data_operação
data = datetime.strptime(data_str, '%Y-%m-%d')
data_menos_um_dia = data - timedelta(days=1)

# Converta o resultado de volta para o formato desejado
data_menos_um_dia_operação = data_menos_um_dia.strftime('%Y-%m-%d')
data_menos_um_dia_operação

'2024-02-26'

In [7]:
data1 = pd.to_datetime(df['Data Vencimento'])
data2 = pd.to_datetime(data_operação)
df['dias_venc'] = (data1 - data2)
df['dias_venc'] = (df['dias_venc'].dt.days)-1


In [8]:
data_cdi = sgs.get(('CDI', 12), last=3000)
data_cdi = data_cdi['CDI']
data_cdi

Date
2012-03-26    0.035984
2012-03-27    0.035984
2012-03-28    0.035948
2012-03-29    0.035948
2012-03-30    0.036093
                ...   
2024-03-01    0.041957
2024-03-04    0.041957
2024-03-05    0.041957
2024-03-06    0.041957
2024-03-07    0.041957
Name: CDI, Length: 3000, dtype: float64

In [9]:
di_per_list = []

for a, b in zip(df['Data Emissão'], df['Taxa de Emissão']):
    data_filt = data_cdi.loc[a:data_menos_um_dia_operação]
    di = ((data_filt/100)+1)*((1+b)**(1/252))
    di_per = di.product()
    di_per_list.append(di_per)

df['di_acum'] = di_per_list

In [10]:
# URL da página
url = 'https://www2.bmf.com.br/pages/portal/bmfbovespa/boletim1/TxRef1.asp'

# Realiza a solicitação HTTP
response = requests.get(url)

# Verifica se a solicitação foi bem-sucedida (código de status 200)
if response.status_code == 200:
    # Lê a tabela HTML diretamente usando o pandas
    dfs = pd.read_html(response.text, flavor='bs4')  # 'bs4' indica o uso do BeautifulSoup para análise HTML

    # O resultado será uma lista de DataFrames, onde cada DataFrame representa uma tabela na página
    # Dependendo da página, você pode precisar procurar qual DataFrame contém os dados desejados
    for i, b in enumerate(dfs):
        print(f"DataFrame {i + 1}:\n{b}")
else:
    print(f"Falha na solicitação. Código de status: {response.status_code}")

di_tabela = dfs[2]
di_tabela = di_tabela.set_index(0)
di_tabela = di_tabela.iloc[2:]
di_tabela = di_tabela[1]
di_tabela = di_tabela.astype(float)
di_tabela.index = di_tabela.index.astype(int)
di_tabela = di_tabela/100


<ipython-input-10-c1f5a03fc16b>:10: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  dfs = pd.read_html(response.text, flavor='bs4')  # 'bs4' indica o uso do BeautifulSoup para análise HTML


DataFrame 1:
                                                   0            1
0                          Atualizado em: 07/03/2024  Retroativo:
1  Taxa: Ajuste cupom  Alumínio  DI x Anbid  Anbi...          NaN
DataFrame 2:
                                                     0          1         2
0    Dias Corridos DI x pré 252(2)(4) 360(1) 1 11,1...        NaN       NaN
1                                        Dias Corridos   DI x pré  DI x pré
2                                        Dias Corridos  252(2)(4)    360(1)
3                                                    1       1115       000
4                                                    6       1100      1045
..                                                 ...        ...       ...
255                                               9476       1075      1052
256                                               9657       1075      1052
257                                              10800       1075      1053
258             

In [11]:
df_di_tabela = pd.DataFrame(di_tabela)


In [12]:
def interpolar_valor(df, x):
  # Encontrar a linha na tabela onde o valor X está presente ou próximo.
  linha = df.index[df.index <= x].max()

  # Se o valor X já estiver presente na tabela DI, retornar o valor correspondente.
  if linha == x:
    return round(df.loc[linha,1],2)

  # Se o valor X não estiver presente na tabela DI, interpolar.
  if linha != x:
    di_anterior = df_di_tabela.loc[linha,1]
    posicao = df_di_tabela.index.get_loc(linha)
    # Obter o próximo índice
    proximo_indice = posicao + 1
    di_prox = df_di_tabela[1].iloc[proximo_indice]
    prox_num_dias_di = df_di_tabela.index[proximo_indice]
    interpolação = di_anterior + (x-linha) * ((di_prox-di_anterior) / (prox_num_dias_di-linha))
    return round(interpolação,2)

  return 0

In [13]:
di_dia_list = []

for a in df['dias_venc']:
  tx_di = interpolar_valor(df_di_tabela, a) /100
  di_dia_list.append(tx_di)

df['di_dia'] = di_dia_list
df['pu_par'] = df['PU Inicial'] * df['di_acum']


In [14]:
num_dias_uteis = []

for a in df['Data Emissão']:
    datas = pd.date_range(start=a, end=data_operação, freq='B')
    dias_uteis = datas[~datas.isin(feriados)]
    dias_uteis_b = len(dias_uteis)-1
    num_dias_uteis.append(dias_uteis_b)


df['num_dias_uteis'] = num_dias_uteis
num_dias_uteis_oper_venc = []

for a in df['Data Vencimento']:
    datas = pd.date_range(start=data_operação, end=a, freq='B')
    dias_uteis = datas[~datas.isin(feriados)]
    dias_uteis_b = len(dias_uteis)-1
    num_dias_uteis_oper_venc.append(dias_uteis_b)


df['num_dias_uteis_oper'] = num_dias_uteis_oper_venc


In [15]:
df['di_int_acum'] =((((1+df['di_dia'])**(1/252))*((1+df['Taxa de Emissão'])**(1/252)))**(252))-1
df['di_int_acum_2'] = +(1+df['di_int_acum'])**(df['num_dias_uteis_oper']/252)-1
df['neg_di_acum']=((((1+df['Taxa de Venda'])**(1/252))*((1+df['di_dia'])**(1/252)))**(252))-1
df['neg_di_acum_2'] =+(1+df['neg_di_acum'])**(df['num_dias_uteis_oper']/252)-1
df['vf'] = (df['di_int_acum_2']+1) * df['pu_par']
df['pu_final'] = df['vf'] / (1+ df['neg_di_acum_2'])

In [16]:
df_filtro = df[['Tipo Aplic.', 'Data Inicial', 'Emissor', 'Banco',
       'Valor Inicial', 'Taxa COMPRA', 'Transação', 'Titulo',
       'Data Emissão', 'Data Vencimento', 'PU Inicial', 'Taxa de Emissão', 'Taxa de Venda','pu_final','Quantidade']]

In [17]:
df_filtro

,Tipo Aplic.,Data Inicial,Emissor,Banco,Valor Inicial,Taxa COMPRA,Transação,Titulo,Data Emissão,Data Vencimento,PU Inicial,Taxa de Emissão,Taxa de Venda,pu_final,Quantidade
0,LF,2023-04-18,SANTANDER,SANTANDER,4000000,0.008,1000000007375,LF0021005P4,2021-05-06,2024-05-06,50000,0.00750,0.003,68410.467389,75
1,LF,2023-05-08,ITAU,ITAU,4000000,0.008,1000000007437,LF0022009MZ,2022-03-28,2024-06-03,100000,0.00658,0.002,128031.004921,41
2,LF,2023-05-08,ITAU,ITAU,4000000,0.008,1000000007438,LF0022001K1,2022-01-21,2024-06-03,100000,0.00667,0.002,130459.081083,40
3,LF,2023-07-26,SICREDI,SICREDI,4000000,0.015,1000000007519,LF002200J39,2022-07-07,2024-07-08,150000,0.01150,0.004,187340.620032,27
4,LF,2023-08-02,SICREDI,SICREDI,4000000,0.015,1000000007534,LF0021009WQ,2021-07-22,2024-07-22,150000,0.01550,0.004,208266.161503,25


In [18]:
carteira = pd.read_excel('/content/Carteira.xlsm')
carteira

,Empresa,Tipo Aplic.,Data Inicial,Data Posicao,Data de Carência,Data de Vencimento,Fundo,Emissor,Nº ID,Transação,Banco,Valor Aplicado
0,1000,LF,2023-07-26,2024-02-27,08/07/2024,2024-07-08,Letra Financeira,SICREDI,NaN,1000000007519,BANCO BRADESCO SA,5000000
1,1000,TF1,2023-05-08,2024-02-27,03/06/2024,2024-06-03,Letra Financeira elegível II,ITAU,080523_TF11,1000000007437,BANCO BRADESCO SA,5000000
2,1000,TF1,2023-05-08,2024-02-27,03/06/2024,2024-06-03,Letra Financeira elegível II,ITAU,080523_TF12,1000000007438,BANCO BRADESCO SA,5000000
3,1000,TL2,2023-04-18,2024-02-27,06/05/2024,2024-05-06,Letra de Financeira indexada,SANTANDER,180423_TL2,1000000007375,BANCO BRADESCO SA,5000000
4,1000,TL2,2023-08-02,2024-02-27,22/07/2024,2024-07-22,Letra de Financeira indexada,SICREDI,020823_TL2,1000000007534,XP INVEST CORRETORA DE CAMBIO SA,5000000


In [19]:
resultado = pd.merge(df_filtro, carteira[['Transação','Valor Aplicado']], on='Transação', how='left')
resultado['Valor Mercado'] = resultado['pu_final'] * resultado['Quantidade']
resultado['Ganho/Perda'] = resultado['Valor Mercado'] - resultado['Valor Aplicado']
resultado['Ganho/Perda %'] = resultado['Ganho/Perda'] / resultado['Valor Aplicado']
resultado

,Tipo Aplic.,Data Inicial,Emissor,Banco,Valor Inicial,Taxa COMPRA,Transação,Titulo,Data Emissão,Data Vencimento,PU Inicial,Taxa de Emissão,Taxa de Venda,pu_final,Quantidade,Valor Aplicado,Valor Mercado,Ganho/Perda,Ganho/Perda %
0,LF,2023-04-18,SANTANDER,SANTANDER,4000000,0.008,1000000007375,LF0021005P4,2021-05-06,2024-05-06,50000,0.00750,0.003,68410.467389,75,5000000,5.130785e+06,130785.054201,0.026157
1,LF,2023-05-08,ITAU,ITAU,4000000,0.008,1000000007437,LF0022009MZ,2022-03-28,2024-06-03,100000,0.00658,0.002,128031.004921,41,5000000,5.249271e+06,249271.201744,0.049854
2,LF,2023-05-08,ITAU,ITAU,4000000,0.008,1000000007438,LF0022001K1,2022-01-21,2024-06-03,100000,0.00667,0.002,130459.081083,40,5000000,5.218363e+06,218363.243339,0.043673
3,LF,2023-07-26,SICREDI,SICREDI,4000000,0.015,1000000007519,LF002200J39,2022-07-07,2024-07-08,150000,0.01150,0.004,187340.620032,27,5000000,5.058197e+06,58196.740867,0.011639
4,LF,2023-08-02,SICREDI,SICREDI,4000000,0.015,1000000007534,LF0021009WQ,2021-07-22,2024-07-22,150000,0.01550,0.004,208266.161503,25,5000000,5.206654e+06,206654.037564,0.041331


In [20]:
nome_arquivo_excel2 = 'LF_cdi+.xlsx'
resultado.to_excel(nome_arquivo_excel2, index=False)

In [21]:
# Fazer o download do arquivo Excel
files.download(nome_arquivo_excel2)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>